In [253]:
import csv
import pandas as pd

# Crawled restaurant data (instagram)
df_=pd.read_csv("jeju_restaurant (3).csv")
df_1=pd.read_csv("jeju_restaurant1.csv")
df_2=pd.read_csv("jeju_restaurant2.csv")
df_3=pd.concat([df_,df_1,df_2])
df_fin=df_3.groupby("name",as_index=False).sum().sort_values(by='mention',ascending=False)
del df_fin['type']
lst = df_fin.values.tolist()
print("Result 0")
print(f"Not Found: {len(lst):>3} rows")

#File 1 (소상공인시장진흥공단_상가(상권)정보 제주)
df_file1=pd.read_csv("jeju_final.csv",usecols=['상호명','도로명주소','위도','경도'])
file1=df_file1.values.tolist()      

for x in lst:
    cnt = 0
    for item in file1:
        if cnt >= 1:
            break
        if item[0] == x[0]:
            cnt += 1
            x.append(item[1])
            x.append(item[3])
            x.append(item[2])
    if cnt == 0:
        x.append('')

cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("-------------------")
print("Result 1")
print(f"   File 1: {cnt1:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")
df1 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df1

Result 0
Not Found: 287 rows
-------------------
Result 1
   File 1:  97 rows
Not Found: 190 rows
    Total: 287 rows


,name,mention,address,latitude,longitude
0,지구상회,310,제주특별자치도 제주시 애월읍 애월해안로 715,33.484000,126.377380
1,더비치크랩,191,제주특별자치도 서귀포시 안덕면 사계남로 168,33.228200,126.307779
2,제주프리또,45,제주특별자치도 제주시 한림읍 귀덕5길 7,33.440401,126.282325
3,템포 다이닝 라운지,35,,NaN,NaN
4,폴비앙 베이커리 카페,30,,NaN,NaN
...,...,...,...,...,...
282,보말이네,1,제주특별자치도 제주시 애월읍 유수암평화5길 51,33.423511,126.404213
283,복음이네김밥,1,,NaN,NaN
284,복음이네김밥만두,1,제주특별자치도 서귀포시 김정문화로 66,33.250563,126.511879
285,봉기네깡통구이,1,제주특별자치도 서귀포시 성산읍 고성오조로 38,33.446859,126.915247


In [254]:
#File 2 (제주 market)
file2 = []
with open('jeju_market.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        file2.append([row[0], row[12], row[15], row[14]])

f1_cnt = 0
f2_cnt = 0
for x in lst:
    if len(x) == 5:
        f1_cnt += 1
        continue
    cnt = 0
    for item in file2:
        if cnt >= 1:
            break
        if item[0] == x[0]:
            cnt += 1
            f2_cnt += 1
            x[2] = item[1]
            x.append(item[2])
            x.append(item[3])

cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("Result 2")
print(f"   File 1: {f1_cnt:>3} rows")
print(f"   File 2: {f2_cnt:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")

df2 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df2

Result 2
   File 1:  97 rows
   File 2:  21 rows
Not Found: 169 rows
    Total: 287 rows


,name,mention,address,latitude,longitude
0,지구상회,310,제주특별자치도 제주시 애월읍 애월해안로 715,33.484,126.37738
1,더비치크랩,191,제주특별자치도 서귀포시 안덕면 사계남로 168,33.2282,126.307779
2,제주프리또,45,제주특별자치도 제주시 한림읍 귀덕5길 7,33.440401,126.282325
3,템포 다이닝 라운지,35,,None,None
4,폴비앙 베이커리 카페,30,,None,None
...,...,...,...,...,...
282,보말이네,1,제주특별자치도 제주시 애월읍 유수암평화5길 51,33.423511,126.404213
283,복음이네김밥,1,,None,None
284,복음이네김밥만두,1,제주특별자치도 서귀포시 김정문화로 66,33.250563,126.511879
285,봉기네깡통구이,1,제주특별자치도 서귀포시 성산읍 고성오조로 38,33.446859,126.915247


In [255]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    if geo is not None:
        crd = [geo.latitude, geo.longitude]
        return crd
    else:
        return None

def addr_revise(address):
    str_del1 = ','
    str_del2 = '('
    if str_del1 in address:
        address = address[:address.index(str_del1)]
    if str_del2 in address:
        address = address[:address.index(str_del2)]
    return address

In [256]:
#File 3 (제주데이터허브)
file3 = []
with open('jeju_food.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        if row[5].find('폐업') == -1:
            file3.append([row[0], row[13]])

f3_cnt = 0
for x in lst:
    if len(x) == 5:
        continue
    cnt = 0
    for item in file3:
        if cnt >= 1:
            break
        if item[0] == x[0]:
            cnt += 1
            addr = addr_revise(item[1])
            x[2] = addr
            if geocoding(addr) is not None:
                x.append(geocoding(addr)[0])
                x.append(geocoding(addr)[1])
                f3_cnt += 1
cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("Result 3")
print(f"   File 1: {f1_cnt:>3} rows")
print(f"   File 2: {f2_cnt:>3} rows")
print(f"   File 3: {f3_cnt:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")        
df3 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df3

Result 3
   File 1:  97 rows
   File 2:  21 rows
   File 3:   8 rows
Not Found: 161 rows
    Total: 287 rows


,name,mention,address,latitude,longitude
0,지구상회,310,제주특별자치도 제주시 애월읍 애월해안로 715,33.484,126.37738
1,더비치크랩,191,제주특별자치도 서귀포시 안덕면 사계남로 168,33.2282,126.307779
2,제주프리또,45,제주특별자치도 제주시 한림읍 귀덕5길 7,33.440401,126.282325
3,템포 다이닝 라운지,35,,None,None
4,폴비앙 베이커리 카페,30,,None,None
...,...,...,...,...,...
282,보말이네,1,제주특별자치도 제주시 애월읍 유수암평화5길 51,33.423511,126.404213
283,복음이네김밥,1,,None,None
284,복음이네김밥만두,1,제주특별자치도 서귀포시 김정문화로 66,33.250563,126.511879
285,봉기네깡통구이,1,제주특별자치도 서귀포시 성산읍 고성오조로 38,33.446859,126.915247


In [257]:
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import time

import re

driver = webdriver.Chrome()
driver.implicitly_wait(3)

def switch_frame(frame):
    driver.switch_to.default_content()
    driver.switch_to.frame(frame)

def remove_after_number(str):
    result = re.sub(r'(\d)\s.*', r'\1', str)
    return result

crawl = []
for e in lst:
    if len(e) == 3:
        crawl.append([lst.index(e),e[0]])
crawl_data = []
f4_cnt = 0
for r in range(len(crawl)):
    driver.get('https://map.naver.com/')
    time.sleep(3)
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(crawl[r][1])
    search.send_keys(Keys.ENTER)

    time.sleep(1)
    switch_frame('searchIframe')
    time.sleep(3)

    res_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
    print(f'{len(res_list)}개의 검색 결과가 나왔습니다.')

    for data in range(len(res_list)):
        if len(res_list) >= 10:
            break
        if len(res_list) == 1:
            switch_frame('entryIframe')
            addr_button = driver.find_elements(By.CSS_SELECTOR, 'div.vV_z_ > a')
            addr_button.__getitem__(0).click()
            time.sleep(2)
            name_path = '/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/span[1]'
            name = driver.find_element(By.XPATH, name_path).text
            addr = driver.find_elements(By.CSS_SELECTOR, 'div.vV_z_ > div.Y31Sf > div.nQ7Lh')
            road = addr.__getitem__(0).text
            road_addr = road[3:(len(road) - 2)]
            if road_addr.startswith('제주'):
                real_addr = remove_after_number(road_addr)
                lst[crawl[r][0]][2] = real_addr
                if geocoding(real_addr) is not None:
                    lst[crawl[r][0]].append(geocoding(real_addr)[0])
                    lst[crawl[r][0]].append(geocoding(real_addr)[1])
                    print(f"Crawl: {name}, Name: {crawl[r][1]}")
                    lst[crawl[r][0]][0] = name
                    print(lst[crawl[r][0]])
                    f4_cnt += 1
        else:
            name_ = driver.find_elements(By.CSS_SELECTOR, 'span.YwYLL')
            name = name_.__getitem__(data).text
            addr_button = driver.find_elements(By.CSS_SELECTOR, 'span.lWwyx > a')
            addr_button.__getitem__(data).click()
            time.sleep(2)
            addr = driver.find_elements(By.CSS_SELECTOR, 'div > div.AbTyi > div.zZfO1')
            road = addr.__getitem__(0).text
            road_addr = road[3:(len(road) - 2)]
            if road_addr.startswith('제주'):
                real_addr = remove_after_number(road_addr)
                lst[crawl[r][0]][2] = real_addr
                if geocoding(real_addr) is not None:
                    lst[crawl[r][0]].append(geocoding(real_addr)[0])
                    lst[crawl[r][0]].append(geocoding(real_addr)[1])
                    print(f"Crawl: {name}, Name: {crawl[r][1]}")
                    lst[crawl[r][0]][0] = name
                    print(lst[crawl[r][0]])
                    f4_cnt += 1
                    break
            addr_button.__getitem__(data).click()

        time.sleep(5)

df4 = pd.DataFrame(lst, columns=['name', 'mention', 'address', 'latitude', 'longitude'])
df4


1개의 검색 결과가 나왔습니다.
Crawl: 템포 다이닝 라운지, Name: 템포 다이닝 라운지
['템포 다이닝 라운지', 35, '제주 서귀포시 안덕면 신화역사로304번길 98', 33.3052583, 126.3149549]
1개의 검색 결과가 나왔습니다.
1개의 검색 결과가 나왔습니다.
Crawl: 사과당 제주점, Name: 사과당 제주점
['사과당 제주점', 22, '제주 제주시 오현길 88', 33.5103409, 126.5261349]
0개의 검색 결과가 나왔습니다.
6개의 검색 결과가 나왔습니다.
Crawl: 이츠버거, Name: 이츠버거
['이츠버거', 10, '제주 제주시 서광로29길 8', 33.5034082, 126.5268105]
7개의 검색 결과가 나왔습니다.
Crawl: 제주한판횟집 제주산방산본점, Name: 제주산방산본점
['제주한판횟집 제주산방산본점', 7, '제주 서귀포시 안덕면 사계로114번길 87', 33.245981, 126.3060353]
10개의 검색 결과가 나왔습니다.
1개의 검색 결과가 나왔습니다.
Crawl: 돗담, Name: 돗담
['돗담', 5, '제주 제주시 연동13길 38', 33.4877302, 126.4945004]
1개의 검색 결과가 나왔습니다.
Crawl: 까망돼지 서귀포점, Name: 까망돼지 서귀포점
['까망돼지 서귀포점', 5, '제주 서귀포시 일주동로 8524', 33.2551463, 126.5775949]
1개의 검색 결과가 나왔습니다.
Crawl: 제주 연북로해물탕 본점, Name: 연북로 해물탕 본점
['제주 연북로해물탕 본점', 5, '제주 제주시 연북로 221', 33.4914841, 126.5463042]
1개의 검색 결과가 나왔습니다.
Crawl: 돼지촌 제주본점, Name: 돼지촌 제주본점
['돼지촌 제주본점', 4, '제주 제주시 원노형남1길 28', 33.4839253, 126.485429]
1개의 검색 결과가 나왔습니다.
0개의 검색 결과가 나왔습니다.
2개의 검색 결과가 나왔

,name,mention,address,latitude,longitude
0,지구상회,310,제주특별자치도 제주시 애월읍 애월해안로 715,33.484,126.37738
1,더비치크랩,191,제주특별자치도 서귀포시 안덕면 사계남로 168,33.2282,126.307779
2,제주프리또,45,제주특별자치도 제주시 한림읍 귀덕5길 7,33.440401,126.282325
3,템포 다이닝 라운지,35,제주 서귀포시 안덕면 신화역사로304번길 98,33.305258,126.314955
4,폴비앙 베이커리 카페,30,제주 제주시 애월읍 고성남서길 8,None,None
...,...,...,...,...,...
282,보말이네,1,제주특별자치도 제주시 애월읍 유수암평화5길 51,33.423511,126.404213
283,복음이네김밥만두,1,제주 서귀포시 김정문화로 64,33.2514,126.509046
284,복음이네김밥만두,1,제주특별자치도 서귀포시 김정문화로 66,33.250563,126.511879
285,봉기네깡통구이,1,제주특별자치도 서귀포시 성산읍 고성오조로 38,33.446859,126.915247


In [258]:
cnt1 = 0
cnt2 = 0
for e in lst:
    if len(e) == 5:
        cnt1 += 1
    else:
        cnt2 += 1
print("Result 4")
print(f"   File 1: {f1_cnt:>3} rows")
print(f"   File 2: {f2_cnt:>3} rows")
print(f"   File 3: {f3_cnt:>3} rows")
print(f"    Crawl: {f4_cnt:>3} rows")
print(f"Not Found: {cnt2:>3} rows")
print(f"    Total: {cnt1+cnt2:>3} rows")

final_lst = []
for i in lst:
    final_lst.append(i)
print(final_lst)

Result 4
   File 1:  97 rows
   File 2:  21 rows
   File 3:   8 rows
    Crawl:  60 rows
Not Found: 101 rows
    Total: 287 rows
[['지구상회', 310, '제주특별자치도 제주시 애월읍 애월해안로 715', 33.4840001976835, 126.377379932118], ['더비치크랩', 191, '제주특별자치도 서귀포시 안덕면 사계남로 168', 33.228199671299, 126.307778963187], ['제주프리또', 45, '제주특별자치도 제주시 한림읍 귀덕5길 7', 33.4404009009907, 126.282324640151], ['템포 다이닝 라운지', 35, '제주 서귀포시 안덕면 신화역사로304번길 98', 33.3052583, 126.3149549], ['폴비앙 베이커리 카페', 30, '제주 제주시 애월읍 고성남서길 8'], ['사과당 제주점', 22, '제주 제주시 오현길 88', 33.5103409, 126.5261349], ['맘앤타르트', 19, '제주특별자치도 제주시 애월읍 중산간서로 6008', 33.4507129866862, 126.373795045802], ['해저', 16, '제주특별자치도 제주시 한림읍 한림해안로 582', 33.4403472318629, 126.276508114726], ['제주세원이네', 11, '제주특별자치도 제주시 삼성로 69', 33.5081409739923, 126.532571581521], ['제주 흑우', 10, ''], ['이츠버거', 10, '제주 제주시 서광로29길 8', 33.5034082, 126.5268105], ['순천미향제주산방산본점', 9, '제주특별자치도 서귀포시 안덕면 사계남로216번길 24-73', 33.2357024, 126.3124913], ['제주공항그때그집', 8, '제주특별자치도 제주시 서해안로 352-1', 33.5103725597917, 126.482

In [266]:
final_result = []
final_result.append(['name', 'mention', 'address', 'latitude', 'longitude', 'type'])
for e in final_lst:
    if len(e) == 5:
        e.append(1)
        print(e)
        final_result.append(e)
print(len(final_result))

with open('output1.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(final_result)

['지구상회', 310, '제주특별자치도 제주시 애월읍 애월해안로 715', 33.4840001976835, 126.377379932118, 1]
['더비치크랩', 191, '제주특별자치도 서귀포시 안덕면 사계남로 168', 33.228199671299, 126.307778963187, 1]
['제주프리또', 45, '제주특별자치도 제주시 한림읍 귀덕5길 7', 33.4404009009907, 126.282324640151, 1]
['템포 다이닝 라운지', 35, '제주 서귀포시 안덕면 신화역사로304번길 98', 33.3052583, 126.3149549, 1]
['사과당 제주점', 22, '제주 제주시 오현길 88', 33.5103409, 126.5261349, 1]
['맘앤타르트', 19, '제주특별자치도 제주시 애월읍 중산간서로 6008', 33.4507129866862, 126.373795045802, 1]
['해저', 16, '제주특별자치도 제주시 한림읍 한림해안로 582', 33.4403472318629, 126.276508114726, 1]
['제주세원이네', 11, '제주특별자치도 제주시 삼성로 69', 33.5081409739923, 126.532571581521, 1]
['이츠버거', 10, '제주 제주시 서광로29길 8', 33.5034082, 126.5268105, 1]
['순천미향제주산방산본점', 9, '제주특별자치도 서귀포시 안덕면 사계남로216번길 24-73', 33.2357024, 126.3124913, 1]
['제주공항그때그집', 8, '제주특별자치도 제주시 서해안로 352-1', 33.5103725597917, 126.482821804665, 1]
['제주정지', 7, '제주특별자치도 제주시 서해안로 480', 33.5189140047842, 126.490497783971, 1]
['제주한판횟집 제주산방산본점', 7, '제주 서귀포시 안덕면 사계로114번길 87', 33.245981, 126.3060353, 1]
['돗담', 